<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/06_autoGen/03_Reflection_and_Blogpost_Writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reflection and Blogpost Writing

## Setup

In [ ]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [ ]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''

## Create a writer agent

In [ ]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [ ]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [ ]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Embark on a transformative journey with DeepLearning.AI, a gateway to the realm of artificial intelligence and deep learning. Offering expert-led courses, this platform equips learners with the skills to navigate the complexities of neural networks and algorithms. From computer vision to natural language processing, DeepLearning.AI unravels the enigma of AI through comprehensive yet accessible modules. Elevate your understanding of machine learning under the guidance of industry pioneers such as Andrew Ng. Ignite your passion for technology and innovation - dive into DeepLearning.AI today.


## Adding reflection

Create a critic agent to reflect on the work of the writer agent.

In [ ]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [ ]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Embark on a transformative journey with DeepLearning.AI, a gateway to the realm of artificial intelligence and deep learning. Offering expert-led courses, this platform equips learners with the skills to navigate the complexities of neural networks and algorithms. From computer vision to natural language processing, DeepLearning.AI unravels the enigma of AI through comprehensive yet accessible modules. Elevate your understanding of machine learning under the guidance of industry pioneers such as Andrew Ng. Ignite your passion for technology and innovation - dive into DeepLearning.AI today.

--------------------------------------------------------------------------------
Critic (to Wri

## Nested chat

In [ ]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [ ]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [ ]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [ ]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [ ]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]

In [ ]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [ ]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Embark on a transformative journey with DeepLearning.AI, a gateway to the realm of artificial intelligence and deep learning. Offering expert-led courses, this platform equips learners with the skills to navigate the complexities of neural networks and algorithms. From computer vision to natural language processing, DeepLearning.AI unravels the enigma of AI through comprehensive yet accessible modules. Elevate your understanding of machine learning under the guidance of industry pioneers such as Andrew Ng. Ignite your passion for technology and innovation - dive into DeepLearning.AI today.

--------------------------------------------------------------------------------

*************


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

After aggregating the feedback from the reviewers, it is evident that there are key areas for improvement in the content related to optimizing for SEO, legal considerations, and ethical concerns.

1. SEO Reviewer:
- Incorporate relevant keywords.
- Utilize structured data.
- Build internal links to improve SEO rankings.

2. Legal Reviewer:
- Consider reframing the title to avoid potential misleading claims.
- Ensure descriptions of the platform align with actual offerings to prevent false advertising.
- Accurately represent industry pioneers' association with DeepLearning.AI and comply with permissions.

3. Ethics Reviewer:
- Avoid exaggerated claims about the platform's capabilities.
- Verify accuracy in descriptions of offerings and industry pioneers.
- Ensure references to individuals are correctly associated and comply with necessary permissions.

Final Suggestions for the 

## Get the summary

In [ ]:
print(res.summary)

Title: Exploring the Potential of DeepLearning.AI

Embark on a transformative journey with DeepLearning.AI, a premier platform for mastering artificial intelligence and deep learning. Offering expert-led courses, this comprehensive hub empowers learners with essential neural network and algorithm skills in areas such as computer vision and natural language processing. Under the guidance of industry leaders like Andrew Ng, ignite your passion for innovation. Incorporating relevant keywords and structured data, optimize your learning experience. To ensure accuracy and compliance, depict the platform's offerings truthfully and represent industry pioneers appropriately. Dive into DeepLearning.AI today to unlock the possibilities of AI responsibly.
